In [ ]:
data_dir = ".\\data\\"

# SLC-2445, vac, 72hrs
filename = "2018-07-27-14-45-18.tdms"
keyence_filename= None

# SLC-1730, vac, 24hrs
filename = "2018-07-26-14-39-24.tdms"
keyence_filename= None

# SLC-1730, vac, 48hrs
filename = "2018-07-30-16-19-44.tdms"
keyence_filename= None

In [ ]:
import batch_utils
import data_tools

In [ ]:
from ipywidgets import FloatProgress
from IPython.display import display

import numpy as np
from pprint import pprint
import datetime
from time import time

from importlib import reload
reload(batch_utils)
reload(data_tools)

data, info = batch_utils.load_data(filename, keyence_filename, suffix="seq") 
pprint(info)
batch_utils.print_lengths(data, "Dataset:")

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

plotter = Scatter


def plot(l=None):
    if l is not None:
        limits = [data['set_point']['time'][l[0]], data['set_point']['time'][l[1]]]
        d = dict(zip(data.keys(), [data_tools.truncate(limits, data[k], find_mode={'seek':False}) for k in data]))
        text = ["i={}".format(i+l[0]) for i in range(len(d['set_point']['time']))]
    else:
        d = data
        text = ["i={}".format(i) for i in range(len(d['set_point']['time']))]

    plots = [
        plotter(x=d['set_point']['time'], y=d['set_point']['data'], name="Set Point", line={'shape':'hv'}, text=text),
        plotter(x=d['position']['time'],  y=d['position']['data'], name="Stage Position"),
        plotter(x=d['position']['time'],  y=np.ma.MaskedArray(d['position']['data'], 
                                                              mask=d['position']['mask']), name="Stage Position Masked"),
        plotter(x=d['position']['time'], y=d['position']['set_point'], name="Stage Error"),
        plotter(x=d['position']['time'], y=np.ma.MaskedArray(d['position']['set_point'], 
                                                             mask=d['position']['mask']), name="Stage Error Masked"),
        plotter(x=d['set_point']['time'], y=d['set_point']['avg_position'], name="Stage Error Average", line={'shape':'hv'}, text=text),
        #plotter(x=d['set_point']['time'], y=d['set_point']['max_position'], name="Stage Error Max"),
        #plotter(x=d['set_point']['time'], y=d['set_point']['min_position'], name="Stage Error Min"),

        Histogram(y=d['set_point']['avg_position'], name="Stage Error Average", xaxis='x2'),
        
        plotter(x=d['temperature']['time'], y=d['temperature']['stage'], name="Stage",   yaxis='y2'),
        plotter(x=d['temperature']['time'], y=d['temperature']['base'],  name="Base",    yaxis='y2'),
        plotter(x=d['temperature']['time'], y=d['temperature']['amb'],   name="Ambient", yaxis='y2'),

    ]

    # Too much data, don't plot the un-aggregated stuff!
    if sum([len(d[e]['time']) for e in d]) > 7000:
        plots = [plots[5], plots[6], plots[7]]

    layout = Layout(
        width = 800,
        height = 800,
        title = filename,
        xaxis = dict(
            title = "Time",
            domain=[0, 0.7],
        ),
        yaxis = dict(
            title = "Position (mm)",
        ),
        xaxis2 = dict(
            title = "Log Prob",
            type = "log",
            domain=[0.8, 1],
        ),
        yaxis2 = dict(
            title = "Temperature (°C)",
            overlaying='y',
            side='right',
        ),
        legend = dict(
            orientation="h",
        )
    )

    fig = Figure(data=plots, layout=layout)
    iplot(fig, filename=filename)

In [ ]:
plot()

In [ ]:

if filename == "2018-07-27-14-45-18.tdms":
    plot([37100, 37200]) #72hr
    
if filename == "2018-07-30-16-19-44.tdms":
    plot([69000, 69100]) #48hr
    

In [ ]:
if filename == "2018-07-27-14-45-18.tdms":
    plot([48550, 48700]) #72hr
    
if filename == "2018-07-30-16-19-44.tdms":
    plot([75600, 75700]) #48hr

In [ ]:
def plot_time(k=None, l=None):
    if l is not None:
        limits = [data['set_point']['time'][l[0]], data['set_point']['time'][l[1]]]
        d = dict(zip(data.keys(), [data_tools.truncate(limits, data[k], find_mode={'seek':False}) for k in data]))
        print("Using limits - don't trust i!")
    else:
        d = data

    if k is None:
        plots = [
            plotter(x=data[key]['time'], y=[(b-a).total_seconds() for a, b in zip(data[key]['time'], data[key]['time'][1:])], 
                    name=key, text=["i={}".format(i) for i in range(len(data[key]['time']))]) for key in data
        ]
    else:
        plots = [
            plotter(x=data[k]['time'], y=[(b-a).total_seconds() for a, b in zip(data[k]['time'], data[k]['time'][1:])], 
                    name=k, text=["i={}".format(i) for i in range(len(data[k]['time']))]),
        ]

    layout = Layout(
        width = 800,
        height = 800,
        title = filename,
        xaxis = dict(
            title = "Time",
        ),
        yaxis = dict(
            title = "Δ Time (s)",
        ),
        legend = dict(
            orientation="h",
        )
    )

    fig = Figure(data=plots, layout=layout)
    iplot(fig, filename=filename)

In [ ]:
plot_time(k="set_point")